In [ ]:
import json
import uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 300px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

In [ ]:
import requests
import json

API_MAIN_URL = 'http://127.0.0.1:8000'

CREDENTIALS = ('kurosaki', 'kurosaki++zangetsu')

In [ ]:
URL = '{}/api/geodb/grid/'.format(API_MAIN_URL)

r = requests.get(URL, auth=CREDENTIALS)

RenderJSON(r.json())

In [ ]:
URL = '{}/api/geodb/dtypes/'.format(API_MAIN_URL)

r = requests.get(URL, auth=CREDENTIALS)

RenderJSON(r.json())

In [ ]:
URL = '{}/api/geodb/datasets/'.format(API_MAIN_URL)

r = requests.get(URL, auth=CREDENTIALS)

RenderJSON(r.json())

In [ ]:
URL = '{}/api/geodb/events/'.format(API_MAIN_URL)

r = requests.get(URL, auth=CREDENTIALS)

RenderJSON(r.json())

In [ ]:
from glass.geo.obj.to import shpext_to_boundary

DATASET = 'dem888_10'
extent = '/mnt/d/gwork/fireloc/pseudo_fire/lmt_coimbra.shp'

HEADERS={
    'content-type'  : 'application/json',
    #'Authorization' : TOKEN
}

URL = '{}/api/geodb/getdata/'.format(API_MAIN_URL)

DATA = {
    "dataset" : DATASET,
    "geom"    : shpext_to_boundary(extent).ExportToWkt(),
    "epsg"    : 3763
}

r = requests.post(URL, headers=HEADERS, data=json.dumps(DATA), auth=CREDENTIALS)

RenderJSON(r.json())

In [ ]:
from glass.pys.web import get_file

TOKEN = 'thjoQD'
output = '/mnt/d/gwork/fireloc/xyz/teste.tif'

URL = '{}/api/geodb/download/{}/'.format(API_MAIN_URL, TOKEN)

get_file(URL, output)

### Old CODE ###

In [ ]:
"""
Send data in Excel to database
"""

import json
import pandas as pd
import requests as rqsts
from glass.fm import tbl_to_obj

tbl = r'D:\GoogleDrive\Fireloc\fireloc_db.xlsx'
sht = 'realtable'
url = "https://vgi.mat.uc.pt/api/sdi/datasets/"

data = tbl_to_obj(tbl, sheet=sht)
data.where(pd.notnull(data), None, inplace=True)
data = data.to_dict(orient='records')

for r in data:
    dd = json.dumps(r)
    response = rqsts.post(url, data=dd, headers={'content-type' : 'application/json'},
                         auth=('kurosaki', 'zangetsu'))

In [ ]:
"""
Get Cells Related with extent
"""

shp = '/home/jasp/xyz/lmt_9.shp'
dataset = 'dem888_10'

REFGRID_TBL = {
    "TBL" : "sdi_refgrid", "GEOM" : "geom",
    "CELLID" : "cellid"
}

con_db = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'expvgi_api'
}

gpkg_fld = '/home/jasp/xpto/expvgi-inescc/api/sdi/sdidata/'

import os
from glass.g.to       import shpext_to_boundary


# Get Extent Geometry
lmt_geom = shpext_to_boundary(shp, out_srs=3763).ExportToWkt()

# Get Intersecting CELLS
Q = (
    "SELECT {ci} FROM ("
        "SELECT {ci}, (ST_Area(ST_Intersection({gc}, ST_GeomFromText('{wkt}', 3763))) "
        "/ ST_Area({gc}) * 100) AS i_area "
        "FROM {t} WHERE ST_Intersects({gc}, ST_GeomFromText('{wkt}', 3763))"
    ") AS foo WHERE i_area > 1"
).format(
    ci=REFGRID_TBL["CELLID"], gc=REFGRID_TBL["GEOM"], t=REFGRID_TBL["TBL"],
    wkt=lmt_geom
)

cells = q_to_obj(con_db, Q)

# Get Geopackage name
gpkg = q_to_obj(con_db, "SELECT rst_gpkg FROM sdi_datasets WHERE slug='{}'".format(
    dataset
)).rst_gpkg.tolist()[0]
print(gpkg)

# Export data from geopackage
rsts = [gpkgrst_to_rst(
    os.path.join(gpkg_fld, gpkg + '.gpkg'),
    "cell__{}".format(str(row.cellid)),
    os.path.join(gpkg_fld, gpkg + '_{}.tif'.format(str(row.cellid)))
) for idx, row in cells.iterrows()]

# Create new Mosaic
rst_data = rsts_to_mosaic(rsts, os.path.join(gpkg_fld, "rst_mos.tif"), api='rasterio')

In [ ]:
from glass.web.ff import get_file_post
from glass.g.to   import shpext_to_boundary
import json

shp = r'C:\gwork\ecgi_val\lmt_9.shp'
dataset = 'dem888_30'

# Get Extent Geometry
lmt_geom = shpext_to_boundary(shp, out_srs=3763).ExportToWkt()

data = {
    'geom' : lmt_geom, 'dataset' : dataset
}

status, ff = get_file_post(
    'http://127.0.0.1:8000/api/sdi/down-dataset/',
    r'C:\gwork\ecgi_val\teste2.tif', data,
    ('kurosaki', 'zangetsu')
)

In [ ]:
import requests
import json

r = requests.post(
    'http://127.0.0.1:8000/api/sdi/teste/',
    data=json.dumps({
        "geometry": "POLYGON ((-59191.4075000012 96083.7674000022 0,808.592499998806 96083.7674000022 0,808.592499998806 36083.7674000022 0,-59191.4075000012 36083.7674000022 0,-59191.4075000012 96083.7674000022 0))", "dataset": "dem888_50"
    }),
    headers={'content-type' : 'application/json'}
)